# Implementing Softmax for the Uninitiated 

The softmax classification function (aka multiclass logistic regression or maxent classifier in traditional machine learning) is a essential building block in deep learning networks. But often we are using softmax without asking how it works in detail (or we are afraid to ask). 
In this notebook, we will show a detailed step-by-step derivation of the update rule for a simple one-layer softmax network with mini-batch stochastic gradient descent.

## The Softmax Function

The [softmax function](http://en.wikipedia.org/wiki/Softmax_function) is the generalization of the [logistic function](http://en.wikipedia.org/wiki/Logistic_function) to multiple categories. The softmax function takes an $c$-dimensional input vector $\mathbf{z}$ and outputs a $c$-dimensional vector of non-negative, real values which can be interpreted as a multi-class categorical probability distribution. 
The softmax function is a normalized exponential and is defined as:

$$
y_k = \sigma_k(\mathbf{z}) = \frac{e^{z_k}}{\sum_{i=1}^{N} e^{z_i}}
$$

The denominator $Z = \sum_{i=1}^{N} e^{z_k}$ acts as a normalizer that ensures that the values sum to unity $ \sum_{k=1}^{N} y_k = 1$. We can interpret $y_k$ as the probability of class $k$ given the input $\mathbf{z}$:

$$
P(y = k | \mathbf{z}) = \sigma_k(\mathbf{z}) = \frac{e^{z_k}}{Z}
$$.

Let's go ahead and implement the softmax function in Python. Note that for computational efficience, we compute the softmax over a complete ''minibatch'' of size ``n_batch`` instead of a single input substract the maximum value in $\mathbf{z}$ for numerical stability ([see this blog post for details](https://lingpipe-blog.com/2009/03/17/softmax-without-overflow/)).|

In [17]:
# python imports
import numpy as np

In [19]:
# define softmax

def softmax(self, Z):
    """Compute softmax activations      
    Z is a matrix with dimension (n_batch, n_classes)                                                            
    """
    ex = np.exp(Z-np.max(Z, axis=1, keepdims=True))
    return ex / ex.sum(axis=1, keepdims=True)

## Derivative of the Softmax Function

To train a neural network, we need to be able to compute the gradient updates of each layer with respect to its inputs. So let's start by computing the partial derivative of the softmax function with respect to a single input $z_i$ using the quotient rule. We distinguish two cases.

\begin{eqnarray}
\text{if}\ k = i &:&\frac{\partial y_i}{\partial z_i} = \frac{\partial \frac{e^{z_i}}{Z}} {\partial z_i} = \frac{e^{z_i} Z - e^{z_i} e^{z_i}} {Z^2} = \frac{e^{z_i}} {Z} \frac{Z - e^{z_i}} {Z} = \frac{e^{z_i}} {Z} (1 - \frac{e^{z_i}} {Z}) = y_i (1-y_i) \\
\text{if}\ k \neq i &:& \frac{\partial y_k}{\partial z_i} = \frac{\partial \frac{e^{z_k}}{Z}} {\partial z_i} = \frac{0 - e^{z_k} e^{z_i}} {Z^2} = - \frac{e^{z_k}} {Z} \frac{e^{z_i}} {Z} = - y_k y_i
\end{eqnarray}


## Cross-entropy cost function 
When we use the softmax function as the output layer in a neural network  classifier, we want to minimize the difference between the predicted output and the true labels $\mathbf{t}$. 
The loss function to minimize is the [cross entropy](https://en.wikipedia.org/wiki/Cross_entropy).

$$
\mathcal{L}(\mathbf{t}, \mathbf{y}) = -\sum_{i=1}^{N} t_i \log{y_i}
$$

For multiclass classification, there will be exactly one $t_i = 1$ and the cross entropy loss is equivalent to the log loss, the negative log-likelihood of the correct class under the model. The cross entropy loss over a minibatch of $M$ examples is:

$$
\mathcal{L}(T, Y) = \mathcal{L}(\mathbf{t_j}, \mathbf{y_j}) =  \sum_{j=1}^{M}\sum_{i=1}^{N} t_{ji} \log{y_{ji}}
$$


## Derivative of the cross-entropy cost function  
Now let's compute the derivative for the cross-entropy loss with respoect to its inputs. Using the chain rule and the derivative for the softmax function from above, we get:

\begin{eqnarray}
\frac{\partial \mathcal{L}(\mathbf{t}, \mathbf{y})}{\partial z_i} &=& - \sum_{j=1}^{N} \frac{\partial t_j \log y_j}{\partial z_i} = -\sum_{j=1}^{N} t_j \frac{\partial \log y_j}{\partial z_i} 
= - \sum_{j=1}^{N} t_j \frac{1}{y_j} \frac{\partial y_j}{\partial z_i} = - \frac{t_i}{y_i} \frac{\partial y_i}{z_i} - \sum_{j\neq i} \frac{t_j}{y_j} \frac{\partial y_j}{\partial z_i} \\
&=& - \frac{t_i}{y_i} y_i(1-y_i) -\sum_{j\neq i} \frac{t_j}{y_i} (-y_j y_i) = -t_i + t_i y_i + \sum_{j\neq i} \frac{t_j}{y_i} =  -t_i  + \sum_{j=1}^{N} t_j y_i = -t_i  + y_i \sum_{j=1}^{N} t_j \\
&=& y_i - t_i 
\end{eqnarray}

The result is surprisingly simple, the derivative of the cross entropy loss with respects to its inputs is just the differenc between the output activation and the true class. 


## Weight update rule 
So far, we have started from the inputs to the softmax layer $\mathbf{z}$ which is a vector of dimension $N$. These inputs in turn, are the result of the previous layer's outputs (or raw features input) $\mathbf{x}$ multiplied by a weight matrix $W \in \mathbb{R}^{N \times D}$:

$$
\mathbf{z} = W \mathbf{x}.
$$

The weight matrix $W$ are the parameters of the softmax layer which we can tune to train the model. We note that $z_i = W_i \mathbf{x}$ where $W_i$ is the $i$-th row in matrix $W$ and that $z_i$ is fully determined by row $W_i$ and indepdendent of the other rows in $W$.
To find the training update rule, we need to find the gradient of the loss function with respect to the weights. Using the chain rule, we get:

$$
\frac{\partial \mathcal{L}(\mathbf{t}, \mathbf{y})}{\partial W_{i}} = 
\frac{\partial \mathcal{L}(\mathbf{t}, \mathbf{y})}{\partial z_{i}} \frac{\partial z_i}{\partial W_{i}} =
 (y_i - t_i)  \mathbf{x}
$$

Again, the result is surprisingly simple, the weight update is difference between the activation and the true class, mutiplied by the input $\mathbf{x}$.

Now, let's see how we can put all this into code. We start with a function that computes the cross entropy loss.

In [ ]:
def loss(self, X, Y, reg):
    """compute cross-entropy loss
    
        X is a feature matrix of dimension (n_batch, n_features)
        Y is a matrix of the true class labels of dimension (n_batch, n_classes)                                                
        """ 
    # compute softmax activations
    Yh = self.predict_lob_proba(X)
    # cross entrupy loss 
    np.log(Yh)
    loss = np.sum(-n[p.log(Yh)
                             [np.arange(X.shape[0]), Y]))
            # add regularization penalty                                                                                 
            loss += reg * np.sum(np.square(self.W))
            return loss / X.shape[0]


### Acknowledgement 
This  excellent [blog post on Peter's notes blog](http://peterroelants.github.io/posts/neural_network_implementation_intermezzo02/) was a great help for the derivation of the math and for how to present the subject in an accessbile and clear way. 